In [2]:
"""
US License Plate Generator - With Real State-Specific Naming Features
=========================================================================

Key Features

1. Format Differences:
   - California: 1ABC234 (digit first)
   - New York/Georgia/Michigan etc: ABC1234 or ABC-1234
   - Maryland: 1AB2345 (unique digit-letter-digit pattern)
   - Delaware/Rhode Island: Pure numeric
   - New Hampshire: 7-digit pure numeric

2. Letter Restrictions:
   - Texas/Tennessee: NO VOWELS (A,E,I,O,U) and no Q
   - Most states: Skip I, O, Q (easily confused with numbers)
   - California: I, O, Q only used in middle position

3. County Code Systems:
   - Wyoming: 1-23 county code prefix
   - Montana: 1-56 county code prefix
   - Idaho: Special county codes (1A, 2B, 1C, etc.)
   - Alabama/South Dakota: County code + letter + digits

4. Special Encoding:
   - Hawaii: First letter indicates island (H/Z=Hawaii County, K=Kauai, M/L=Maui, others=Honolulu)
   - Massachusetts: Last digit indicates expiration month (1-9=Jan-Sep, 0=Oct)

5. First Character Ranges:
   - California: First digit 1-9 (indicates registration era)
   - Some states start letters from specific characters
"""

import random
import pandas as pd
from datetime import datetime

# ============================================================
# Letter Set Definitions
# ============================================================

# Standard letters (excluding I, O, Q - easily confused with numbers)
STANDARD_LETTERS = 'ABCDEFGHJKLMNPRSTUVWXYZ'

# Texas/Tennessee letters (no vowels A,E,I,O,U and no Q)
NO_VOWEL_LETTERS = 'BCDFGHJKLMNPRSTVWXYZ'

# California middle position letters (can include I, O, Q)
CA_MIDDLE_LETTERS = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

# California edge position letters (excluding I, O, Q)
CA_EDGE_LETTERS = 'ABCDEFGHJKLMNPRSTUVWXYZ'

# Hawaii island-specific first letters
HI_HAWAII_COUNTY = ['H', 'Z']  # Big Island
HI_KAUAI_COUNTY = ['K']  # Kauai
HI_MAUI_COUNTY = ['M', 'L']  # Maui
HI_HONOLULU_LETTERS = 'ABCDEFGJNPRSTUVWXY'  # Honolulu (excludes H,K,L,M,Z,I,O,Q)

# Wyoming county codes (1-23)
WYOMING_COUNTIES = list(range(1, 24))

# Montana county codes (1-56)
MONTANA_COUNTIES = list(range(1, 57))

# Idaho county codes
IDAHO_COUNTY_CODES = [
    '1A', '2A', '1B', '2B', '3B', '4B', '5B', '6B', '7B', '8B',
    '1C', '2C', '3C', '4C', '5C', '6C', '7C', '8C', '9C',
    '1E', '1F', '2F', '1G', '2G', '3G',
    '1J', '2J', '1K', '1L', '2L', '3L', '4L', '5L',
    '1M', '1N', '2N', '1P', '2P', '3P',
    '1S', '2S', '1T', '2T', '1V', '1W'
]

# Alabama county codes (1-67)
ALABAMA_COUNTIES = list(range(1, 68))

# South Dakota county codes (1-67)
SD_COUNTIES = list(range(1, 68))

# Nebraska large county 3-letter codes
NEBRASKA_LARGE_COUNTIES = ['DOG', 'LAN', 'SAR', 'DAK', 'SCO', 'BUF', 'HAL', 'LIN', 'MAD', 'PLT']
NEBRASKA_COUNTY_NUMBERS = list(range(1, 94))


# ============================================================
# Helper Functions
# ============================================================

def get_random_letter(letter_set=STANDARD_LETTERS):
    """Get a random letter from the specified letter set"""
    return random.choice(letter_set)


def get_random_digit(exclude_zero=False):
    """Generate a random digit (0-9 or 1-9)"""
    if exclude_zero:
        return str(random.randint(1, 9))
    return str(random.randint(0, 9))


def get_random_digits(count, exclude_zero_first=False):
    """Generate multiple random digits"""
    result = ''
    for i in range(count):
        if i == 0 and exclude_zero_first:
            result += get_random_digit(exclude_zero=True)
        else:
            result += get_random_digit()
    return result


def get_random_letters(count, letter_set=STANDARD_LETTERS):
    """Generate multiple random letters"""
    return ''.join(random.choice(letter_set) for _ in range(count))


# ============================================================
# State-Specific Generator Functions
# ============================================================

def generate_california():
    """
    California: 1ABC234 format
    - First digit 1-9 (indicates registration era)
    - Middle 3 letters, I/O/Q only allowed in middle position
    - Last 3 digits
    """
    first_digit = get_random_digit(exclude_zero=True)  # 1-9
    # Middle letters: first and third cannot use I,O,Q; second can
    letter1 = get_random_letter(CA_EDGE_LETTERS)
    letter2 = get_random_letter(CA_MIDDLE_LETTERS)  # Can include I,O,Q
    letter3 = get_random_letter(CA_EDGE_LETTERS)
    last_digits = get_random_digits(3)
    return f"{first_digit}{letter1}{letter2}{letter3}{last_digits}"


def generate_texas():
    """
    Texas: ABC-1234 or LLL-DDDD format
    - NO VOWELS (A,E,I,O,U) and no Q
    - This is a key distinguishing feature!
    """
    letters = get_random_letters(3, NO_VOWEL_LETTERS)
    digits = get_random_digits(4)
    if random.random() < 0.5:
        return f"{letters}-{digits}"
    return f"{letters}{digits}"


def generate_tennessee():
    """
    Tennessee: ABC-1234 or A12-34B format
    - NO VOWELS - same as Texas
    - This is a key distinguishing feature!
    """
    if random.random() < 0.6:
        # Standard format
        letters = get_random_letters(3, NO_VOWEL_LETTERS)
        digits = get_random_digits(4)
        return f"{letters}-{digits}"
    else:
        # A12-34B format
        l1 = get_random_letter(NO_VOWEL_LETTERS)
        d1 = get_random_digits(2)
        d2 = get_random_digits(2)
        l2 = get_random_letter(NO_VOWEL_LETTERS)
        return f"{l1}{d1}-{d2}{l2}"


def generate_hawaii():
    """
    Hawaii: ABC-123 format
    - First letter indicates island county
    - H/Z = Hawaii County (Big Island)
    - K = Kauai County
    - M/L = Maui County
    - Other letters = Honolulu (most common)
    """
    # Weight by population (Honolulu is largest)
    county_weights = [0.70, 0.15, 0.08, 0.07]  # Honolulu, Hawaii, Maui, Kauai
    county = random.choices(['honolulu', 'hawaii', 'maui', 'kauai'], weights=county_weights)[0]

    if county == 'honolulu':
        first_letter = random.choice(HI_HONOLULU_LETTERS)
    elif county == 'hawaii':
        first_letter = random.choice(HI_HAWAII_COUNTY)
    elif county == 'maui':
        first_letter = random.choice(HI_MAUI_COUNTY)
    else:  # kauai
        first_letter = random.choice(HI_KAUAI_COUNTY)

    # Remaining letters use standard set (excluding I,O,Q)
    other_letters = get_random_letters(2, STANDARD_LETTERS)
    digits = get_random_digits(3)
    return f"{first_letter}{other_letters}{digits}"


def generate_massachusetts():
    """
    Massachusetts: Multiple formats
    - Last digit indicates expiration month (1=Jan, ..., 9=Sep, 0=Oct)
    - Formats: 1ABC23, 123ABC, AB1234, etc.
    """
    month_digit = str(random.randint(0, 9))  # 0-9 corresponds to month

    format_type = random.choice(['1ABC23', '123AB4', 'AB1234', '1AB234'])

    if format_type == '1ABC23':
        d1 = get_random_digit()
        letters = get_random_letters(3, STANDARD_LETTERS)
        d2 = get_random_digit()
        return f"{d1}{letters}{d2}{month_digit}"
    elif format_type == '123AB4':
        digits = get_random_digits(3)
        letters = get_random_letters(2, STANDARD_LETTERS)
        return f"{digits}{letters}{month_digit}"
    elif format_type == 'AB1234':
        letters = get_random_letters(2, STANDARD_LETTERS)
        digits = get_random_digits(3)
        return f"{letters}{digits}{month_digit}"
    else:  # 1AB234
        d1 = get_random_digit()
        letters = get_random_letters(2, STANDARD_LETTERS)
        digits = get_random_digits(2)
        return f"{d1}{letters}{digits}{month_digit}"


def generate_maryland():
    """
    Maryland: 1AB2345 format (unique digit-letter-digit pattern)
    - This format is unique to Maryland
    """
    d1 = get_random_digit(exclude_zero=True)
    letters = get_random_letters(2, STANDARD_LETTERS)
    digits = get_random_digits(4)
    return f"{d1}{letters}{digits}"


def generate_delaware():
    """
    Delaware: Pure numeric format (1-6 digits)
    - Low numbers are highly valuable
    """
    # Most are 6 digits
    if random.random() < 0.85:
        return get_random_digits(6)
    else:
        length = random.randint(4, 5)
        return get_random_digits(length)


def generate_rhode_island():
    """
    Rhode Island: Pure numeric format (5-6 digits)
    """
    if random.random() < 0.7:
        return get_random_digits(6)
    else:
        return get_random_digits(5)


def generate_new_hampshire():
    """
    New Hampshire: 7-digit pure numeric
    - Only state with exactly 7 numeric digits
    """
    return get_random_digits(7)


def generate_wyoming():
    """
    Wyoming: County code (1-23) + '-' + 5 digits
    - County codes are based on historical population ranking
    """
    county = random.choice(WYOMING_COUNTIES)
    digits = get_random_digits(5)
    return f"{county}-{digits}"


def generate_montana():
    """
    Montana: County code (1-56) + '-' + 5 digits + 1 letter
    - County codes assigned around 1930
    """
    county = random.choice(MONTANA_COUNTIES)
    digits = get_random_digits(5)
    letter = get_random_letter(STANDARD_LETTERS)
    return f"{county}-{digits}{letter}"


def generate_idaho():
    """
    Idaho: County code + space + letter + digits
    - County codes like 1A, 2B, 1C based on county names
    """
    county = random.choice(IDAHO_COUNTY_CODES)
    letter = get_random_letter(STANDARD_LETTERS)
    if random.random() < 0.5:
        digits = get_random_digits(5)
        return f"{county} {letter}{digits}"
    else:
        digits = get_random_digits(4)
        end_letter = get_random_letter(STANDARD_LETTERS)
        return f"{county} {letter}{digits}{end_letter}"


def generate_alabama():
    """
    Alabama: County code (1-67) + letter + 4 digits + letter
    """
    county = random.choice(ALABAMA_COUNTIES)
    l1 = get_random_letter(STANDARD_LETTERS)
    digits = get_random_digits(4)
    l2 = get_random_letter(STANDARD_LETTERS)
    return f"{county}{l1}{digits}{l2}"


def generate_south_dakota():
    """
    South Dakota: County code (1-67) + letter + 3 digits
    """
    county = random.choice(SD_COUNTIES)
    letter = get_random_letter(STANDARD_LETTERS)
    digits = get_random_digits(3)
    return f"{county}{letter}{digits}"


def generate_nebraska():
    """
    Nebraska: Large counties use 3-letter code, small counties use numeric code
    """
    if random.random() < 0.3:
        # Large county 3-letter code
        county = random.choice(NEBRASKA_LARGE_COUNTIES)
        digits = get_random_digits(3)
        return f"{county}{digits}"
    else:
        # Numeric county code
        county = random.choice(NEBRASKA_COUNTY_NUMBERS)
        letter = get_random_letter(STANDARD_LETTERS)
        digits = get_random_digits(3)
        return f"{county}-{letter}{digits}"


def generate_connecticut():
    """
    Connecticut: AB-12345 or AB12345 format
    """
    letters = get_random_letters(2, STANDARD_LETTERS)
    digits = get_random_digits(5)
    if random.random() < 0.5:
        return f"{letters}-{digits}"
    return f"{letters}{digits}"


def generate_illinois():
    """
    Illinois: AB 12345 or AB-12345 format
    """
    letters = get_random_letters(2, STANDARD_LETTERS)
    digits = get_random_digits(5)
    if random.random() < 0.5:
        return f"{letters} {digits}"
    return f"{letters}-{digits}"


def generate_new_jersey():
    """
    New Jersey: A12-BCD or A12BCD format
    """
    l1 = get_random_letter(STANDARD_LETTERS)
    digits = get_random_digits(2)
    letters = get_random_letters(3, STANDARD_LETTERS)
    if random.random() < 0.5:
        return f"{l1}{digits}-{letters}"
    return f"{l1}{digits}{letters}"


def generate_florida():
    """
    Florida: ABC-D12 or LLLL-DD format
    """
    letters1 = get_random_letters(3, STANDARD_LETTERS)
    letter2 = get_random_letter(STANDARD_LETTERS)
    digits = get_random_digits(2)
    if random.random() < 0.5:
        return f"{letters1}-{letter2}{digits}"
    return f"{letters1}{letter2}{digits}"


def generate_colorado():
    """
    Colorado: ABC-123 or 123-ABC format (with hyphen)
    """
    letters = get_random_letters(3, STANDARD_LETTERS)
    digits = get_random_digits(3)
    if random.random() < 0.5:
        return f"{letters}-{digits}"
    return f"{digits}-{letters}"


def generate_arkansas():
    """
    Arkansas: ABC-12D format
    """
    letters = get_random_letters(3, STANDARD_LETTERS)
    digits = get_random_digits(2)
    letter = get_random_letter(STANDARD_LETTERS)
    return f"{letters}-{digits}{letter}"


def generate_arizona():
    """
    Arizona: "Alphabet Soup" - 6-character mixed format
    - 4th position is always a digit
    """
    chars = []
    for i in range(6):
        if i == 3:  # 4th position is always a digit
            chars.append(get_random_digit())
        else:
            if random.random() < 0.5:
                chars.append(get_random_letter(STANDARD_LETTERS))
            else:
                chars.append(get_random_digit())
    return ''.join(chars)


def generate_nevada():
    """
    Nevada: 123A45 format (1 letter, 5 digits)
    """
    d1 = get_random_digits(3)
    letter = get_random_letter(STANDARD_LETTERS)
    d2 = get_random_digits(2)
    return f"{d1}{letter}{d2}"


def generate_dc():
    """
    Washington DC: AB-1234 or AB1234 format
    """
    letters = get_random_letters(2, STANDARD_LETTERS)
    digits = get_random_digits(4)
    if random.random() < 0.5:
        return f"{letters}-{digits}"
    return f"{letters}{digits}"


def generate_kansas():
    """
    Kansas: 1234ABC format
    """
    digits = get_random_digits(4)
    letters = get_random_letters(3, STANDARD_LETTERS)
    return f"{digits}{letters}"


def generate_utah():
    """
    Utah: A12BCD format
    """
    l1 = get_random_letter(STANDARD_LETTERS)
    digits = get_random_digits(2)
    letters = get_random_letters(3, STANDARD_LETTERS)
    return f"{l1}{digits}{letters}"


def generate_maine():
    """
    Maine: 1234AB format
    """
    digits = get_random_digits(4)
    letters = get_random_letters(2, STANDARD_LETTERS)
    return f"{digits}{letters}"


def generate_standard_abc1234(use_dash=False):
    """
    Standard ABC1234 format (used by NY, GA, MI, NC, OH, PA, VA, WA, WI, etc.)
    """
    letters = get_random_letters(3, STANDARD_LETTERS)
    digits = get_random_digits(4)
    if use_dash and random.random() < 0.3:
        return f"{letters}-{digits}"
    return f"{letters}{digits}"


def generate_standard_abc123():
    """
    Standard ABC123 format (used by AK, IA, IN, KY, LA, MN, ND, OK, OR, SC, VT, etc.)
    """
    letters = get_random_letters(3, STANDARD_LETTERS)
    digits = get_random_digits(3)
    return f"{letters}{digits}"


def generate_standard_123abc():
    """
    Standard 123ABC format
    """
    digits = get_random_digits(3)
    letters = get_random_letters(3, STANDARD_LETTERS)
    return f"{digits}{letters}"


# ============================================================
# State Configuration
# ============================================================

STATE_CONFIG = {
    'AL': {'name': 'Alabama', 'generator': generate_alabama},
    'AK': {'name': 'Alaska', 'generator': generate_standard_abc123},
    'AZ': {'name': 'Arizona', 'generator': generate_arizona},
    'AR': {'name': 'Arkansas', 'generator': generate_arkansas},
    'CA': {'name': 'California', 'generator': generate_california},
    'CO': {'name': 'Colorado', 'generator': generate_colorado},
    'CT': {'name': 'Connecticut', 'generator': generate_connecticut},
    'DE': {'name': 'Delaware', 'generator': generate_delaware},
    'DC': {'name': 'Washington DC', 'generator': generate_dc},
    'FL': {'name': 'Florida', 'generator': generate_florida},
    'GA': {'name': 'Georgia', 'generator': lambda: generate_standard_abc1234(use_dash=True)},
    'HI': {'name': 'Hawaii', 'generator': generate_hawaii},
    'ID': {'name': 'Idaho', 'generator': generate_idaho},
    'IL': {'name': 'Illinois', 'generator': generate_illinois},
    'IN': {'name': 'Indiana', 'generator': generate_standard_abc123},
    'IA': {'name': 'Iowa', 'generator': generate_standard_abc123},
    'KS': {'name': 'Kansas', 'generator': generate_kansas},
    'KY': {'name': 'Kentucky', 'generator': generate_standard_abc123},
    'LA': {'name': 'Louisiana', 'generator': generate_standard_abc123},
    'ME': {'name': 'Maine', 'generator': generate_maine},
    'MD': {'name': 'Maryland', 'generator': generate_maryland},
    'MA': {'name': 'Massachusetts', 'generator': generate_massachusetts},
    'MI': {'name': 'Michigan', 'generator': lambda: generate_standard_abc1234(use_dash=True)},
    'MN': {'name': 'Minnesota', 'generator': generate_standard_abc123},
    'MS': {'name': 'Mississippi', 'generator': generate_standard_abc123},
    'MO': {'name': 'Missouri', 'generator': generate_standard_abc123},
    'MT': {'name': 'Montana', 'generator': generate_montana},
    'NE': {'name': 'Nebraska', 'generator': generate_nebraska},
    'NV': {'name': 'Nevada', 'generator': generate_nevada},
    'NH': {'name': 'New Hampshire', 'generator': generate_new_hampshire},
    'NJ': {'name': 'New Jersey', 'generator': generate_new_jersey},
    'NM': {'name': 'New Mexico', 'generator': generate_standard_abc123},
    'NY': {'name': 'New York', 'generator': lambda: generate_standard_abc1234(use_dash=True)},
    'NC': {'name': 'North Carolina', 'generator': lambda: generate_standard_abc1234(use_dash=True)},
    'ND': {'name': 'North Dakota', 'generator': generate_standard_abc123},
    'OH': {'name': 'Ohio', 'generator': lambda: generate_standard_abc1234(use_dash=True)},
    'OK': {'name': 'Oklahoma', 'generator': generate_standard_abc123},
    'OR': {'name': 'Oregon', 'generator': generate_standard_123abc},
    'PA': {'name': 'Pennsylvania', 'generator': lambda: generate_standard_abc1234(use_dash=True)},
    'RI': {'name': 'Rhode Island', 'generator': generate_rhode_island},
    'SC': {'name': 'South Carolina', 'generator': generate_standard_abc123},
    'SD': {'name': 'South Dakota', 'generator': generate_south_dakota},
    'TN': {'name': 'Tennessee', 'generator': generate_tennessee},
    'TX': {'name': 'Texas', 'generator': generate_texas},
    'UT': {'name': 'Utah', 'generator': generate_utah},
    'VT': {'name': 'Vermont', 'generator': generate_standard_abc123},
    'VA': {'name': 'Virginia', 'generator': lambda: generate_standard_abc1234(use_dash=True)},
    'WA': {'name': 'Washington', 'generator': lambda: generate_standard_abc1234(use_dash=True)},
    'WV': {'name': 'West Virginia', 'generator': generate_standard_abc123},
    'WI': {'name': 'Wisconsin', 'generator': lambda: generate_standard_abc1234(use_dash=True)},
    'WY': {'name': 'Wyoming', 'generator': generate_wyoming},
}


# ============================================================
# Main Functions
# ============================================================

def generate_plates_for_state(state_code, num_plates=10000):
    """Generate license plate data for a specific state"""
    config = STATE_CONFIG[state_code]
    generator = config['generator']
    state_name = config['name']

    plates = []
    for _ in range(num_plates):
        plate_number = generator()
        plates.append({
            'plate_number': plate_number,
            'state_code': state_code,
            'state_name': state_name
        })

    return plates


def generate_all_plates(plates_per_state=10000):
    """Generate license plate data for all states"""
    all_plates = []
    states = list(STATE_CONFIG.keys())

    print(f"Starting data generation: {len(states)} states x {plates_per_state:,} plates = {len(states)*plates_per_state:,} total")
    print("-" * 60)

    for i, state in enumerate(states, 1):
        state_plates = generate_plates_for_state(state, plates_per_state)
        all_plates.extend(state_plates)

        if i % 10 == 0 or i == len(states):
            print(f"[{i}/{len(states)}] Completed {state} ({STATE_CONFIG[state]['name']}) - Total: {len(all_plates):,}")

    print("-" * 60)
    print(f"✅ Total generated: {len(all_plates):,}")

    return all_plates


def save_to_csv(plates, filename='dataset.csv'):
    """Save data to CSV file"""
    df = pd.DataFrame(plates)
    df.to_csv(filename, index=False)
    print(f"✅ Saved to: {filename}")
    return filename


def show_samples(plates, samples_per_state=5):
    """Display sample plates for each state"""
    df = pd.DataFrame(plates)
    print("\n" + "=" * 70)
    print("Sample License Plates by State (Note the unique features)")
    print("=" * 70)

    for state in sorted(df['state_code'].unique()):
        state_df = df[df['state_code'] == state]
        state_name = state_df['state_name'].iloc[0]
        samples = state_df['plate_number'].head(samples_per_state).tolist()
        print(f"{state:2s} ({state_name:20s}): {samples}")


def analyze_distinguishing_features(plates):
    """Analyze distinguishing features by state"""
    df = pd.DataFrame(plates)

    print("\n" + "=" * 70)
    print("License Plate Feature Analysis")
    print("=" * 70)

    print("\n[1] Pure Numeric States:")
    for state in ['DE', 'RI', 'NH']:
        samples = df[df['state_code'] == state]['plate_number'].head(3).tolist()
        print(f"  {state}: {samples}")

    print("\n[2] No Vowel States (Texas/Tennessee) - KEY FEATURE:")
    for state in ['TX', 'TN']:
        samples = df[df['state_code'] == state]['plate_number'].head(3).tolist()
        print(f"  {state}: {samples}")

    print("\n[3] County Code System States:")
    for state in ['WY', 'MT', 'ID', 'AL', 'SD', 'NE']:
        samples = df[df['state_code'] == state]['plate_number'].head(3).tolist()
        print(f"  {state}: {samples}")

    print("\n[4] Island Encoding State (Hawaii):")
    hi_df = df[df['state_code'] == 'HI']
    print(f"  HI: {hi_df['plate_number'].head(5).tolist()}")
    print(f"      First letter codes: H/Z=Hawaii, K=Kauai, M/L=Maui, Others=Honolulu")

    print("\n[5] Unique Format States:")
    for state in ['CA', 'MD', 'MA', 'NJ', 'AZ']:
        samples = df[df['state_code'] == state]['plate_number'].head(3).tolist()
        print(f"  {state}: {samples}")


# ============================================================
# Main Program
# ============================================================

if __name__ == '__main__':
    # Generate data
    plates = generate_all_plates(plates_per_state=10000)

    # Shuffle data
    random.shuffle(plates)

    # Display samples
    show_samples(plates)

    # Analyze features
    analyze_distinguishing_features(plates)

    # Save to CSV
    save_to_csv(plates)



Starting data generation: 51 states x 10,000 plates = 510,000 total
------------------------------------------------------------
[10/51] Completed FL (Florida) - Total: 100,000
[20/51] Completed ME (Maine) - Total: 200,000
[30/51] Completed NH (New Hampshire) - Total: 300,000
[40/51] Completed RI (Rhode Island) - Total: 400,000
[50/51] Completed WI (Wisconsin) - Total: 500,000
[51/51] Completed WY (Wyoming) - Total: 510,000
------------------------------------------------------------
✅ Total generated: 510,000

Sample License Plates by State (Note the unique features)
AK (Alaska              ): ['CHP494', 'TVL456', 'VUR287', 'BTW902', 'BTU464']
AL (Alabama             ): ['9G7304S', '44S3592R', '36N2050U', '58C6025K', '19A9071V']
AR (Arkansas            ): ['DBD-13A', 'DFL-25W', 'EUF-76Y', 'XCU-62M', 'UZC-95N']
AZ (Arizona             ): ['ZK0154', '8X41R3', '1WV621', 'D5W609', 'C8V622']
CA (California          ): ['4HNZ123', '9XPS545', '4UMW699', '4GFJ826', '5LXG950']
CO (Colorado    